Coursera의 ['Deep Learning Specialization'](https://www.coursera.org/specializations/deep-learning#about) 중 Improving Deep Neural Network : Hyperparameter tuning, Regularization and Optimization - Hyperparameter tuning, Batch Normalization and Programing Framworks를 요약한 것

# Hyperparameter Tuning

Neural Network를 학습하는데 있어서 다양한 Hyperparameter를 세팅하는 과정이 개입되어있다.
* Learning Rate : $\alpha$
* Momentum Term : $\beta$
* Adam parameter : $\beta_1, \beta_2, \epsilon$
* No. of Layer, No. of Hidden Unit
* Mini-batch Size
* $\vdots$

Hyperparameter를 설정하고 고르는데 있어 특정 parameter가 더 특별히 중요할 수도 있다. 대부분의 경우에 Learning rate $\alpha$는 매우 중요한 Hyperparameter이다. Hyperparameter를 튜닝할 때에는 Hyperparameter들을 임의로 지정하여 시도해본다. 그 이유는 어떤 Hyperparameter 값이 문제에서 가장 중요한지 미리 알 수 없기 때문이다.
* Try random values(Not uniformly)
* Coarse to fine

## Using an appropriate scale to pick hyperparameters

Hyperparameter를 튜닝하기 위해 임의로 값을 설정할 때, 균일하게 임의로 샘플링 하는 것이 아니라, 적합한 scale을 골라서 Hyperparameter를 탐색해야 한다. 예를 들어 Learning rate $\alpha$를 튜닝할 때, 0.0001부터 1 사이를 균일하게 샘플링 한다면 90%가 0.1에서 1 사이에 있게 된다. 단지 10%만이 0.0001과 0.1 사이에 있기 때문에 좋은 탐색 방법이 아니다. 이런 경우에는 규일하게 샘플링 하는 대신 log scale을 사용하여 탐색을 하는 것이 더 좋은 방법이다.

# Batch Normalization

Batch Normalization은 Hyperparameter Tuning을 더 쉽게 해주고, Neural Network를 더 robust하게 만들어주는 방법이다. 앞서 Input 변수들을 Normalization 해주는 것이 학습 속도를 높이는 방법이라 얘기했었다. 이와같이 각 Layer의 Output($a^{[l]}$)을 Normalization 해주는 것이 $W^{[l+1]}, b^{[l+1]}$를 학습 속도를 높일 수 있다. 각 $a^{[l]}$이 다음 $l+1$ Layer의 입력층이기 때문이다. 정확하게는 Batch Normalization은 $a^{[l]}$이 아닌 $z^{[l]}$을 Normalization 해준다.
$$ \mu = \frac{1}{m}\sum_{i=1}^{m} z^{[l](i)}$$
$$ \sigma^2 = \frac{1}{m} \sum_{i=1}^{m} \left( z^{[l](i)} - \mu \right)^2$$
$$ z_{norm}^{[l](i)} = \frac{z^{[l](i)} - \mu}{\sqrt{\sigma^2 + \epsilon}}$$

Normalization을 해준 $z_{norm}^{[l](i)}$ 평균 0, 분산 1을 갖는다. 그러나 모든 Hidden Unit이 항상 평균 0, 분산 1을 갖는 것보다는 다른 분포도를 갖게 해주고 싶기 때문에 $z_{norm}^{[l](i)}$에 약간의 변형을 더해준다.
$$\tilde{z}^{[l](i)} = \gamma \cdot z_{norm}^{[l](i)} + \beta$$
여기서 $\gamma, \beta$는 학습 가능한 parameters이다.

# Multi-Class Classification

이진 분류가 아닌 다진 분류의 경우 마지막 Output Layer의 Unit 수가 Class의 수이고, 각 Unit은 그 Class가 나올 확률 값을 나타내줘야한다. 네트워크가 이렇게 할 수 있도록 쓰는 일반적인 모델은 Softamx Layer이다.
Output Layer의 z는 다음과 같이 구한다.
$$z^{[L]} = W^{[L]} \cdot a^{[L-1]} + b^{[L]}$$

이제 이 $z^{[L]}$에 Softmax Activation Function을 적용한다. 
$$t = e^{z^{[L]}}\text{, C = No. of Class}$$
$$\text{Shape of }t : (C, 1)$$

$$a^{[L]} = \frac{e^{z^{[L]}}}{\displaystyle \sum_{j=1}^{C} t_i}$$
$$\downarrow$$
$$a^{[L]}_i = \frac{t_i}{\displaystyle \sum_{j=1}^{C} t_i}$$

### 예시
$$z^{[L]} = \begin{bmatrix} 5 \\  2 \\ -1 \\ 3 \end{bmatrix}$$
$$t = \begin{bmatrix} e^5 \\  e^2 \\ e^{-1} \\ 3 \end{bmatrix} = \begin{bmatrix} 148.4 \\  7.4 \\ 0.4 \\ 20.1 \end{bmatrix}$$
$$\displaystyle \sum_{j=1}^{4} t_i = 176.3$$
$$a^{[L]} = \frac{1}{176.3}\times t = \begin{bmatrix} \frac{148.4}{176.3} \\  \frac{7.4}{176.3} \\ \frac{0.4}{176.3} \\ \frac{20.1}{176.3} \end{bmatrix} = \begin{bmatrix} 0.842 \\  0.042 \\ 0.0.002 \\ 0.114 \end{bmatrix}$$

## Loss Function

Softmax Classification에서 주로 사용하는 Loss Function은 다음과 같다.
$$L(\hat{y}, y) = - \displaystyle \sum_{j=1}^{C} y_j\log{\hat{y_j}}$$

여기서 실제 값인 $y$는 해당 클래스에만 1값을 갖고 나머지는 0이기 때문에 실제 Loss를 계산하면 $-log{\hat{y_j}}$만 남게 되고 이 Loss를 최소화하기 위해서는 결국 해당 클래스의 확률 값인 $\hat{y_j}$를 크게 해줘야한다.

$$J(W, b) = \frac{1}{m}\sum_{i=1}^{m} L(\hat{y}^{(i)}, y^{(i)})$$